In [5]:
from pysmt.shortcuts import *
from pysmt.typing import INT

import itertools

def genState(vars,s,i,n):
    state = {}
    for v in vars:
        state[v] = Symbol(v+'!'+s+str(i),INT)
    
    return state


def init(state,n):
    bvmax = Int(2**n)
    return And(GT(state['x'],Int(0)),
               LT(state['x'],bvmax),
               GT(state['y'],Int(0)),
               LT(state['y'],bvmax),
               Equals(state['z'],Int(0)),
               Equals(state['pc'],Int(0)))


def trans(curr,prox,n):
    m = n+1
    bv1 = Int(1)
    bv2 = Int(2)
    bvmax = Int(2**n)
    
    t0 = And(Equals(curr['pc'],Int(0)),
             Equals(prox['pc'],Int(1)),
             Equals(curr['x'],prox['x']),
             Equals(curr['y'],prox['y']),
             Equals(curr['z'],prox['z'])) 
    
    t1 = And(Equals(curr['pc'],Int(1)),
             Equals(even(curr['y']),Int(0)),
             Not(Equals(prox['y'],Int(0))),
             Equals(prox['pc'],Int(2)),
             Equals(curr['x'],prox['x']),
             Equals(curr['y'],prox['y']),
             Equals(curr['z'],prox['z']))
    
    t2 = And(Equals(curr['pc'],Int(2)),
             Equals(prox['pc'],Int(1)),
             GT(bvmax,prox['x']),
             Equals(Times(curr['x'],bv2),prox['x']),
             Equals(Div(curr['y'],bv2),prox['y']),
             Equals(curr['z'],prox['z']))
    
    te = And(Equals(curr['pc'],Int(2)),
             Equals(prox['pc'],Int(5)),
             GE(Times(curr['x'],bv2),bvmax),
             Equals(Times(curr['x'],bv2),prox['x']),
             Equals(curr['y'],prox['y']),
             Equals(curr['z'],prox['z']))
    
    te2 = And(Equals(curr['pc'],Int(5)),
             Equals(prox['pc'],Int(5)),
             Equals(curr['x'],prox['x']),
             Equals(curr['y'],prox['y']),
             Equals(curr['z'],prox['z']))
    
    t3 = And(Equals(curr['pc'],Int(1)),
             Equals(curr['y']2,Int(1)),
             Not(Equals(prox['y'],Int(0))),
             Equals(prox['pc'],Int(3)),
             Equals(curr['x'],prox['x']),
             Equals(curr['y'],prox['y']),
             Equals(curr['z'],prox['z']))
    
    t4 = And(Equals(curr['pc'],Int(3)),
             Equals(prox['pc'],Int(1)),
             Equals(curr['x'],prox['x']),
             Equals(Minus(curr['y'],Int(1)),prox['y']),
             Equals(curr['z']+curr['x'],prox['z']),
             GT(bvmax,prox['z']))
    
    te3 = And(Equals(curr['pc'],Int(3)),
             Equals(prox['pc'],Int(5)),
             Equals(curr['x'],prox['x']),
             Equals(Minus(curr['y'],Int(1)),prox['y']),
             Equals(curr['z']+curr['x'],prox['z']),
             BVUGE(curr['x']+curr['z'],bvmax))
    
    t5 = And(Equals(curr['pc'],Int(1)),
             Equals(prox['pc'],Int(4)),
             Equals(curr['x'],prox['x']),
             Equals(curr['y'],prox['y']),
             Equals(curr['z'],prox['z']),
             Equals(curr['y'],Int(0)))
    
    t6 = And(Equals(curr['pc'],Int(4)),
             Equals(prox['pc'],Int(4)),
             Equals(curr['x'],prox['x']),
             Equals(curr['y'],prox['y']),
             Equals(curr['z'],prox['z']))
    
    return Or(t1,t2,t3,t0,t4,t5,t6,te,te2,te3)



def error(state,n):
    m = n+1
    bvmax = Int(2**n)
    bvmaxx = (2**n+1)
    
    t1 = And(Equals(state['pc'],Int(5)),GE(state['x'],bvmax),GE(state['y'],Int(0)),
             GE(state['z'],Int(0)),LE(state['x'],bvmaxx),LE(state['y'],bvmax),LE(state['z'],bvmax))
    t2 = And(Equals(state['pc'],Int(5)),BVUGE(state['x'],Int(0)),BVUGE(state['y'],Int(0)),
             GE(state['z'],bvmax),LE(state['z'],bvmaxx),LE(state['x'],bvmax),LE(state['y'],bvmax))
    
    return Or(t1,t2)






def genTrace(vars,init,trans,error,k,n):
    with Solver(name="z3") as s:
        
        
        X = [genState(vars,'X',i,k) for i in range(n+1)]   # cria n+1 estados (com etiqueta X)
        I = init(X[0],k)
        Tks = [ trans(X[i],X[i+1],k) for i in range(n) ]
        
        if s.solve([I,And(Tks)]):      # testa se I /\ T^n  é satisfazível
            for i in range(n):
                print("Estado:",i)
                for v in X[i]:
                    print("          ",v,'=',s.get_value(X[i][v]))

In [6]:
genTrace(['pc','x','y','z'],init,trans,error,6,30)

NameError: name 'even' is not defined

In [ ]:
def invert(trans):
    return (lambda c,p,t: trans(p,c,t))

def baseName(s):
    return ''.join(list(itertools.takewhile(lambda x: x!='!', s)))

def rename(form,state):
    vs = get_free_variables(form)
    pairs = [ (x,state[baseName(x.symbol_name())]) for x in vs ]
    return form.substitute(dict(pairs))

def same(state1,state2):
    return And([Equals(state1[x],state2[x]) for x in state1])

In [ ]:
def model_checking(vars,init,trans,error,N,M,k):
    with Solver(name="z3") as s:
        
        # Criar todos os estados que poderão vir a ser necessários.
        X = [genState(vars,'X',i,k) for i in range(N+1)]
        Y = [genState(vars,'Y',i,k) for i in range(M+1)]
        
        # Estabelecer a ordem pela qual os pares (n,m) vão surgir. Por exemplo:
        order = sorted([(a,b) for a in range(1,N+1) for b in range(1,M+1)],key=lambda tup:tup[0]+tup[1]) 
        
        for (n,m) in order:
            
            Tn = And([trans(X[i],X[i+1],k) for i in range(n)])
            I = init(X[0],k)
            Rn = And(I,Tn)
            
            Bm = And([invert(trans)(Y[i],Y[i+1],k) for i in range(m)])
            E = error(Y[0],k)
            Um = And(E,Bm)
            
            Vnm = And(Rn,same(X[n],Y[m]),Um)
            
            print(Vnm)
            
            if s.solve([Vnm]):
                print("unsafe")
                return
            else:
                C = binary_interpolant(And(Rn,same(X[n],Y[m])),Um)
                if C is None:
                    print("interpolante none")
                    break
                C0 = rename(C,X[0])
                C1 = rename(C,X[1])
                T = trans(X[0],X[1])
                
                if not s.solve([C0,T,Not(C1)]):
                    print("safe")
                    return
                else:
                    S = rename(C,X[n])
                    while True:
                        A = And(S,trans(X[n],Y[m]))
                        if s.solve([A,Um]):
                            print("nao é possivel majorar")
                            break
                        else:
                            Cnew = binary_interpolant(A,Um)
                            Cn = rename(Cnew,X[n])
                            if s.solve([Cn,Not(S)]):
                                S = Or(S,Cn)
                            else:
                                print("safe")
                                return

#####

model_checking(['pc','x','y','z'], init, trans, error, 50, 50,6)  